In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

In [ ]:
import torch
from pytorch_pretrained_biggan import (
    BigGAN,
    truncated_noise_sample,
    one_hot_from_int
)
import PIL.Image
import numpy as np
import os
import glob
import argparse
from tqdm import tqdm
import json
import pickle
import matplotlib.pyplot as plt
import utils_bigbigan as ubigbi

model = BigGAN.from_pretrained('biggan-deep-256').cuda()

In [ ]:
def convert_to_images(obj):
    """ Convert an output tensor from BigGAN in a list of images.
    """
    # need to fix import, see: https://github.com/huggingface/pytorch-pretrained-BigGAN/pull/14/commits/68a7446951f0b9400ebc7baf466ccc48cdf1b14c
    if not isinstance(obj, np.ndarray):
        obj = obj.detach().numpy()
    obj = obj.transpose((0, 2, 3, 1))
    obj = np.clip(((obj + 1) / 2.0) * 256, 0, 255)
    img = []
    for i, out in enumerate(obj):
        out_array = np.asarray(np.uint8(out), dtype=np.uint8)
        img.append(PIL.Image.fromarray(out_array))
    return img

In [ ]:
w_zoom = np.load('../../../gan_steerability/w_zoom.npy')


bs = 4
truncation = 1.0
seed = 0
idx_rnd = np.random.randint(1000, size=bs)
idx_rnd = [207, 207, 207, 207]
noise_vector = truncated_noise_sample(truncation=truncation,
                                      batch_size=bs,
                                      seed=seed)


class_vector = one_hot_from_int(idx_rnd, batch_size=bs)
class_vector = torch.from_numpy(class_vector).cuda()
noise_vector = torch.from_numpy(noise_vector + w_zoom).cuda()

# from model
with torch.no_grad():
    output = model(noise_vector, class_vector, truncation=1.0)
output = output.cpu()
ims = convert_to_images(output)
ubigbi.imshow(ubigbi.imgrid(np.stack(ims), cols=4))

In [ ]:
w_zoom = np.load('../../../gan_steerability/w_zoom.npy')